In [ ]:
#http://www.portalaction.com.br/analise-de-regressao/411-modelo-estatistico
#https://towardsdatascience.com/the-logistic-regression-algorithm-75fe48e21cfa

#exemplo: http://nbviewer.jupyter.org/gist/justmarkham/6d5c061ca5aee67c4316471f8c2ae976

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#from  sklearn import datasets

In [2]:
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
#from sklearn.cross_validation import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score, fbeta_score, roc_auc_score, auc, roc_curve, precision_score, recall_score, classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import StratifiedKFold, KFold, LeaveOneOut

C:\Users\gabi_\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
# load dataset
dta = sm.datasets.fair.load_pandas().data
dta.head()

,rate_marriage,age,yrs_married,children,religious,educ,occupation,occupation_husb,affairs
0,3.0,32.0,9.0,3.0,3.0,17.0,2.0,5.0,0.111111
1,3.0,27.0,13.0,3.0,1.0,14.0,3.0,4.0,3.230769
2,4.0,22.0,2.5,0.0,1.0,16.0,3.0,5.0,1.400000
3,4.0,37.0,16.5,4.0,3.0,16.0,5.0,5.0,0.727273
4,5.0,27.0,9.0,1.0,1.0,14.0,3.0,4.0,4.666666


In [13]:
dta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6366 entries, 0 to 6365
Data columns (total 9 columns):
rate_marriage      6366 non-null float64
age                6366 non-null float64
yrs_married        6366 non-null float64
children           6366 non-null float64
religious          6366 non-null float64
educ               6366 non-null float64
occupation         6366 non-null float64
occupation_husb    6366 non-null float64
affairs            6366 non-null float64
dtypes: float64(9)
memory usage: 447.7 KB


In [16]:
a = dta.affairs

In [17]:
a

0        0.111111
1        3.230769
2        1.400000
3        0.727273
4        4.666666
5        4.666666
6        0.852174
7        1.826086
8        4.799999
9        1.333333
10       3.266665
11       2.041666
12       0.484848
13       2.000000
14       3.266665
15       1.361111
16       2.000000
17       1.826086
18       1.826086
19       2.041666
20       7.839996
21       2.545454
22       0.532609
23       0.622222
24       0.583333
25       4.799999
26       0.166667
27       0.615385
28       1.187878
29      11.199999
          ...    
6336     0.000000
6337     0.000000
6338     0.000000
6339     0.000000
6340     0.000000
6341     0.000000
6342     0.000000
6343     0.000000
6344     0.000000
6345     0.000000
6346     0.000000
6347     0.000000
6348     0.000000
6349     0.000000
6350     0.000000
6351     0.000000
6352     0.000000
6353     0.000000
6354     0.000000
6355     0.000000
6356     0.000000
6357     0.000000
6358     0.000000
6359     0.000000
6360     0

In [18]:
# add "affair" column: 1 represents having affairs, 0 represents not
dta['affair'] = (dta.affairs > 0).astype(int)
dta.head()

,rate_marriage,age,yrs_married,children,religious,educ,occupation,occupation_husb,affairs,affair
0,3.0,32.0,9.0,3.0,3.0,17.0,2.0,5.0,0.111111,1
1,3.0,27.0,13.0,3.0,1.0,14.0,3.0,4.0,3.230769,1
2,4.0,22.0,2.5,0.0,1.0,16.0,3.0,5.0,1.400000,1
3,4.0,37.0,16.5,4.0,3.0,16.0,5.0,5.0,0.727273,1
4,5.0,27.0,9.0,1.0,1.0,14.0,3.0,4.0,4.666666,1


In [19]:
from patsy import dmatrices
# create dataframes with an intercept column and dummy variables for
# occupation and occupation_husb
y, X = dmatrices('affair ~ rate_marriage + age + yrs_married + children + \
                  religious + educ + C(occupation) + C(occupation_husb)',
                  dta, return_type="dataframe")

# flatten y into a 1-D array - exigido pela funcao logistic regression



In [30]:
y = np.ravel(y)

In [31]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [32]:
cv_kfold = KFold(n_splits=3, shuffle=False, random_state=None)
cv_loo = LeaveOneOut()

In [33]:
clf = LogisticRegression(random_state= 12)
#LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, 
 #                             fit_intercept=True, intercept_scaling=1, class_weight=None, 
  #                            random_state=None, solver='liblinear', max_iter=100, multi_class='ovr', 
   #                           verbose=0, warm_start=False, n_jobs=1)
    ####multi_class nao funciona pra liblinear (default - 2 classes)

parameters = {
    'penalty' : ['l2'],
    'fit_intercept': [True] ,
    'solver': ['liblinear']
}

#scoring pode ser: accuracy, roc_auc, recall, precision... 
### regressao logisitca nao tem parametros para ficar testando
model_lr = GridSearchCV(estimator = clf, param_grid = parameters, scoring= 'roc_auc', fit_params=None, n_jobs=1, iid=True, refit=True, 
                     cv=cv_kfold, verbose=0, pre_dispatch='2*n_jobs', error_score='raise', return_train_score= True)
model_lr

GridSearchCV(cv=KFold(n_splits=3, random_state=None, shuffle=False),
       error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=12, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [36]:
grid_lr_fit = model_lr.fit(X_train, y_train)

In [ ]:
grid_lr_fit.

In [37]:
modelo = grid_lr_fit.best_estimator_

In [ ]:
modelo.

In [38]:
grid_lr_fit.best_score_

0.7393597309444458

In [41]:
grid_lr_fit.best_params_

{}

In [36]:
pd.DataFrame(grid_lr_fit.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_fit_intercept,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,0.015504,0.006393,0.000667,0.000471,True,l2,liblinear,"{'fit_intercept': True, 'penalty': 'l2', 'solv...",0.76069,0.736092,0.721284,0.73936,0.016253,1,0.736361,0.750817,0.756525,0.747901,0.008486


In [37]:
predict_lr = grid_lr_fit.predict(X_test)

In [39]:
pd.DataFrame(confusion_matrix(y_test, predict_lr))

,0,1
0,993,107
1,316,176


In [41]:
prob_lr = grid_lr_fit.predict_proba(X_test)
prob_lr

array([[0.35103468, 0.64896532],
       [0.90788434, 0.09211566],
       [0.73334233, 0.26665767],
       ...,
       [0.77762896, 0.22237104],
       [0.38614252, 0.61385748],
       [0.36478365, 0.63521635]])

In [42]:
predict_lr

array([1., 0., 0., ..., 0., 1., 1.])

In [43]:
recall_score(y_test, predict_lr)

0.35772357723577236

In [44]:
precision_score(y_test, predict_lr)

0.6219081272084805

In [45]:
f1_score(y_test, predict_lr)

0.45419354838709675

In [46]:
accuracy_score(y_test, predict_lr)

0.7342964824120602

In [47]:
roc_auc_score(y_test, prob_lr[:,1])

0.7481504065040652

In [68]:
print(metrics.classification_report(y_test, predict_lr))

             precision    recall  f1-score   support

        0.0       0.76      0.90      0.82      1100
        1.0       0.62      0.36      0.45       492

avg / total       0.72      0.73      0.71      1592

